In [75]:
import simple_salesforce
from os import environ as env

SF = simple_salesforce.Salesforce(env['SF_USER'], env['SF_PASS'], env['SF_TOKEN'])
print(SF)

In [76]:
lookup = {
    'Name': 'name',
    'Commercial_Demo_Status__c': 'status',
    'BSEED_COM_Final_Grade_Approved__c': 'bseed_com_final_grade_approved',
    'BSEED_COM_Open_Hole_Approved__c': 'bseed_com_open_hole_approved',
    'BSEED_COM_Winter_Grade_Approved__c': 'bseed_com_winter_grade_approved',
    'DBA_Received_EMG_Letter_Date__c': 'dba_received_emg_letter_date',
    'Final_Grade_Approved_Dt__c': 'final_grade_approved_dt',
    'Open_Hole_Approved_Dt__c': 'open_hole_approved_dt',
    'Winter_Grade_Approved_Dt__c': 'winter_grade_approved_dt',
    'Demo_Cost_Abatement__c': 'demo_cost_abatement',
    'Demo_Cost_Knock__c': 'demo_cost_knock',
    'Demo_NtP_Dt__c': 'demo_ntp_dt',
    'Demo_Proj_Demo_Dt__c': 'demo_proj_demo_dt',
    'Demo_Pulled_Date__c': 'demo_pulled_date',
    'ENV_Demo_Proceed_Dt__c': 'env_demo_proceed_dt',
    'ENV_Group_Number__c': 'env_group_number',
    'Knock_Start_Dt__c': 'knock_start_dt',
    'Demolition_Contractor__r.Name': 'demo_contractor',
    'DBA_COM_Property__r.Council_District__c': 'dba_com_property_council_district',
    'DBA_COM_Property__r.Latitude__c': 'dba_com_property_lat',
    'DBA_COM_Property__r.Longitude__c': 'dba_com_property_lng',
    'DBA_COM_Property__r.Name': 'dba_com_property_name',
    'DBA_COM_Property__r.Neighborhood__c': 'dba_com_property_neighborhood',
    'DBA_COM_Property__r.Parcel_ID__c': 'dba_com_property_parcel_id'
}

query = """
Select {} from DBA_Commercial_Demo__c where
    Knock_Start_Dt__c >= 2014-01-01
        OR
    Knock_Start_Dt__c = Null
""".format(",".join(lookup.keys()))

res = SF.query_all(query)

In [77]:
import pandas
df = pandas.DataFrame.from_records(res['records'])
df.rename(columns=lookup, inplace=True)
df.drop('attributes', inplace=True, axis=1)

print(df.shape)
df[:5]

(1019, 19)


,name,status,bseed_com_final_grade_approved,bseed_com_open_hole_approved,bseed_com_winter_grade_approved,dba_received_emg_letter_date,final_grade_approved_dt,open_hole_approved_dt,winter_grade_approved_dt,demo_cost_abatement,demo_cost_knock,demo_ntp_dt,demo_proj_demo_dt,demo_pulled_date,env_demo_proceed_dt,env_group_number,knock_start_dt,Demolition_Contractor__r,DBA_COM_Property__r
0,DBA-CD-1604,Demo Contracted,None,None,None,None,None,None,None,2915.0,86384.15,2017-08-21,None,None,2017-05-18,2017-D,None,"{'attributes': {'type': 'Account', 'url': '/se...","{'attributes': {'type': 'Account', 'url': '/se..."
1,DBA-CD-1652,Demolished,2017-06-05,2017-02-10,2017-02-20,None,None,2017-02-10,2017-02-20,NaN,58150.00,2017-01-30,None,None,None,None,2017-01-30,"{'attributes': {'type': 'Account', 'url': '/se...","{'attributes': {'type': 'Account', 'url': '/se..."
2,DBA-CD-1709,Demo Pipeline,None,None,None,None,None,None,None,NaN,NaN,None,None,None,None,None,None,None,"{'attributes': {'type': 'Account', 'url': '/se..."
3,DBA-CD-1712,Demo Pipeline,None,None,None,None,None,None,None,NaN,NaN,None,None,None,None,None,None,None,"{'attributes': {'type': 'Account', 'url': '/se..."
4,DBA-CD-1713,Demo Pipeline,None,None,None,None,None,None,None,NaN,NaN,None,None,None,None,None,None,None,"{'attributes': {'type': 'Account', 'url': '/se..."


In [78]:
# filter by demo status
df = df.loc[df['status'].isin(['Demo Contracted', 'Demolished', 'Demo Pipeline'])]
df.shape

(972, 19)

In [79]:
# filter for null demo pulled dates
df = df[df['demo_pulled_date'].isnull()]
df.shape

(961, 19)

In [80]:
# add related property account fields
df['address'] = df['DBA_COM_Property__r'].apply(lambda x: x['Name'])
df['parcel_id'] = df['DBA_COM_Property__r'].apply(lambda x: x['Parcel_ID__c'])
df['lat'] = df['DBA_COM_Property__r'].apply(lambda x: x['Latitude__c'])
df['lng'] = df['DBA_COM_Property__r'].apply(lambda x: x['Longitude__c'])
df['neighborhood'] = df['DBA_COM_Property__r'].apply(lambda x: x['Neighborhood__c'] if x['Neighborhood__c'] else "")
df['council_district'] = df['DBA_COM_Property__r'].apply(lambda x: x['Council_District__c'] if x['Council_District__c'] else "")

df[:5]

,name,status,bseed_com_final_grade_approved,bseed_com_open_hole_approved,bseed_com_winter_grade_approved,dba_received_emg_letter_date,final_grade_approved_dt,open_hole_approved_dt,winter_grade_approved_dt,demo_cost_abatement,...,env_group_number,knock_start_dt,Demolition_Contractor__r,DBA_COM_Property__r,address,parcel_id,lat,lng,neighborhood,council_district
0,DBA-CD-1604,Demo Contracted,None,None,None,None,None,None,None,2915.0,...,2017-D,None,"{'attributes': {'type': 'Account', 'url': '/se...","{'attributes': {'type': 'Account', 'url': '/se...",19499 Derby,09024819.,42.436498,-83.103958,Nolan,3
1,DBA-CD-1652,Demolished,2017-06-05,2017-02-10,2017-02-20,None,None,2017-02-10,2017-02-20,NaN,...,None,2017-01-30,"{'attributes': {'type': 'Account', 'url': '/se...","{'attributes': {'type': 'Account', 'url': '/se...",19181 Conant,09008596-620,42.434274,-83.074989,Nolan,3
2,DBA-CD-1709,Demo Pipeline,None,None,None,None,None,None,None,NaN,...,None,None,None,"{'attributes': {'type': 'Account', 'url': '/se...",8451 Southfield,22074530-2,42.355076,-83.216861,Warrendale,7
3,DBA-CD-1712,Demo Pipeline,None,None,None,None,None,None,None,NaN,...,None,None,None,"{'attributes': {'type': 'Account', 'url': '/se...",7310 Southfield,22072794-5,42.345061,-83.214806,Warren Ave Community,7
4,DBA-CD-1713,Demo Pipeline,None,None,None,None,None,None,None,NaN,...,None,None,None,"{'attributes': {'type': 'Account', 'url': '/se...",18633 W Mcnichols,22013191.,42.415184,-83.225047,North Rosedale Park,1


In [81]:
# add related contractor name field
def getContractorName(obj):
    try:
        obj['Name']
        return obj['Name']
    except TypeError:
        return ""

df['demo_contractor_name'] = df['Demolition_Contractor__r'].apply(lambda x: getContractorName(x))
df[:5]

,name,status,bseed_com_final_grade_approved,bseed_com_open_hole_approved,bseed_com_winter_grade_approved,dba_received_emg_letter_date,final_grade_approved_dt,open_hole_approved_dt,winter_grade_approved_dt,demo_cost_abatement,...,knock_start_dt,Demolition_Contractor__r,DBA_COM_Property__r,address,parcel_id,lat,lng,neighborhood,council_district,demo_contractor_name
0,DBA-CD-1604,Demo Contracted,None,None,None,None,None,None,None,2915.0,...,None,"{'attributes': {'type': 'Account', 'url': '/se...","{'attributes': {'type': 'Account', 'url': '/se...",19499 Derby,09024819.,42.436498,-83.103958,Nolan,3,Able Demolition
1,DBA-CD-1652,Demolished,2017-06-05,2017-02-10,2017-02-20,None,None,2017-02-10,2017-02-20,NaN,...,2017-01-30,"{'attributes': {'type': 'Account', 'url': '/se...","{'attributes': {'type': 'Account', 'url': '/se...",19181 Conant,09008596-620,42.434274,-83.074989,Nolan,3,DMC Consultants
2,DBA-CD-1709,Demo Pipeline,None,None,None,None,None,None,None,NaN,...,None,None,"{'attributes': {'type': 'Account', 'url': '/se...",8451 Southfield,22074530-2,42.355076,-83.216861,Warrendale,7,
3,DBA-CD-1712,Demo Pipeline,None,None,None,None,None,None,None,NaN,...,None,None,"{'attributes': {'type': 'Account', 'url': '/se...",7310 Southfield,22072794-5,42.345061,-83.214806,Warren Ave Community,7,
4,DBA-CD-1713,Demo Pipeline,None,None,None,None,None,None,None,NaN,...,None,None,"{'attributes': {'type': 'Account', 'url': '/se...",18633 W Mcnichols,22013191.,42.415184,-83.225047,North Rosedale Park,1,


In [82]:
# add new cols - prioritize bseed inspection approval date, if that's null use demo inspection date
df['open_hole_date'] = df['bseed_com_open_hole_approved'].fillna(df['open_hole_approved_dt'])
df['winter_grade_date'] = df['bseed_com_winter_grade_approved'].fillna(df['winter_grade_approved_dt'])
df['final_grade_date'] = df['bseed_com_final_grade_approved'].fillna(df['final_grade_approved_dt'])
df[:5]

,name,status,bseed_com_final_grade_approved,bseed_com_open_hole_approved,bseed_com_winter_grade_approved,dba_received_emg_letter_date,final_grade_approved_dt,open_hole_approved_dt,winter_grade_approved_dt,demo_cost_abatement,...,address,parcel_id,lat,lng,neighborhood,council_district,demo_contractor_name,open_hole_date,winter_grade_date,final_grade_date
0,DBA-CD-1604,Demo Contracted,None,None,None,None,None,None,None,2915.0,...,19499 Derby,09024819.,42.436498,-83.103958,Nolan,3,Able Demolition,None,None,None
1,DBA-CD-1652,Demolished,2017-06-05,2017-02-10,2017-02-20,None,None,2017-02-10,2017-02-20,NaN,...,19181 Conant,09008596-620,42.434274,-83.074989,Nolan,3,DMC Consultants,2017-02-10,2017-02-20,2017-06-05
2,DBA-CD-1709,Demo Pipeline,None,None,None,None,None,None,None,NaN,...,8451 Southfield,22074530-2,42.355076,-83.216861,Warrendale,7,,None,None,None
3,DBA-CD-1712,Demo Pipeline,None,None,None,None,None,None,None,NaN,...,7310 Southfield,22072794-5,42.345061,-83.214806,Warren Ave Community,7,,None,None,None
4,DBA-CD-1713,Demo Pipeline,None,None,None,None,None,None,None,NaN,...,18633 W Mcnichols,22013191.,42.415184,-83.225047,North Rosedale Park,1,,None,None,None


In [83]:
# sum knock and abatement costs for total demo cost
df['total_demo_cost'] = df['demo_cost_abatement'] + df['demo_cost_knock']
df[:5]

,name,status,bseed_com_final_grade_approved,bseed_com_open_hole_approved,bseed_com_winter_grade_approved,dba_received_emg_letter_date,final_grade_approved_dt,open_hole_approved_dt,winter_grade_approved_dt,demo_cost_abatement,...,parcel_id,lat,lng,neighborhood,council_district,demo_contractor_name,open_hole_date,winter_grade_date,final_grade_date,total_demo_cost
0,DBA-CD-1604,Demo Contracted,None,None,None,None,None,None,None,2915.0,...,09024819.,42.436498,-83.103958,Nolan,3,Able Demolition,None,None,None,89299.15
1,DBA-CD-1652,Demolished,2017-06-05,2017-02-10,2017-02-20,None,None,2017-02-10,2017-02-20,NaN,...,09008596-620,42.434274,-83.074989,Nolan,3,DMC Consultants,2017-02-10,2017-02-20,2017-06-05,NaN
2,DBA-CD-1709,Demo Pipeline,None,None,None,None,None,None,None,NaN,...,22074530-2,42.355076,-83.216861,Warrendale,7,,None,None,None,NaN
3,DBA-CD-1712,Demo Pipeline,None,None,None,None,None,None,None,NaN,...,22072794-5,42.345061,-83.214806,Warren Ave Community,7,,None,None,None,NaN
4,DBA-CD-1713,Demo Pipeline,None,None,None,None,None,None,None,NaN,...,22013191.,42.415184,-83.225047,North Rosedale Park,1,,None,None,None,NaN


In [84]:
df['knock_start_dt'][:15]

0           None
1     2017-01-30
2           None
3           None
4           None
5           None
6           None
7     2016-06-16
8     2016-12-12
9     2014-07-18
11    2016-11-24
12    2016-10-09
13          None
14    2017-06-21
15          None
Name: knock_start_dt, dtype: object

In [85]:
df['demo_proj_demo_dt'][:15]

0           None
1           None
2           None
3           None
4           None
5           None
6           None
7           None
8           None
9           None
11    2016-11-21
12    2016-10-09
13          None
14          None
15          None
Name: demo_proj_demo_dt, dtype: object

In [86]:
# if already knocked down, zero out projected demo dt
df.loc[df.status.isin(['Demolished']), 'demo_proj_demo_dt'] = None
df[:5]

,name,status,bseed_com_final_grade_approved,bseed_com_open_hole_approved,bseed_com_winter_grade_approved,dba_received_emg_letter_date,final_grade_approved_dt,open_hole_approved_dt,winter_grade_approved_dt,demo_cost_abatement,...,parcel_id,lat,lng,neighborhood,council_district,demo_contractor_name,open_hole_date,winter_grade_date,final_grade_date,total_demo_cost
0,DBA-CD-1604,Demo Contracted,None,None,None,None,None,None,None,2915.0,...,09024819.,42.436498,-83.103958,Nolan,3,Able Demolition,None,None,None,89299.15
1,DBA-CD-1652,Demolished,2017-06-05,2017-02-10,2017-02-20,None,None,2017-02-10,2017-02-20,NaN,...,09008596-620,42.434274,-83.074989,Nolan,3,DMC Consultants,2017-02-10,2017-02-20,2017-06-05,NaN
2,DBA-CD-1709,Demo Pipeline,None,None,None,None,None,None,None,NaN,...,22074530-2,42.355076,-83.216861,Warrendale,7,,None,None,None,NaN
3,DBA-CD-1712,Demo Pipeline,None,None,None,None,None,None,None,NaN,...,22072794-5,42.345061,-83.214806,Warren Ave Community,7,,None,None,None,NaN
4,DBA-CD-1713,Demo Pipeline,None,None,None,None,None,None,None,NaN,...,22013191.,42.415184,-83.225047,North Rosedale Park,1,,None,None,None,NaN


In [87]:
df['knock_start_dt'][:15]

0           None
1     2017-01-30
2           None
3           None
4           None
5           None
6           None
7     2016-06-16
8     2016-12-12
9     2014-07-18
11    2016-11-24
12    2016-10-09
13          None
14    2017-06-21
15          None
Name: knock_start_dt, dtype: object

In [88]:
df['demo_proj_demo_dt'][:15]

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
11    None
12    None
13    None
14    None
15    None
Name: demo_proj_demo_dt, dtype: object

In [89]:
df['status'][:15]

0     Demo Contracted
1          Demolished
2       Demo Pipeline
3       Demo Pipeline
4       Demo Pipeline
5       Demo Pipeline
6       Demo Pipeline
7          Demolished
8          Demolished
9          Demolished
11         Demolished
12         Demolished
13      Demo Pipeline
14         Demolished
15      Demo Pipeline
Name: status, dtype: object